In [1]:
import pandas as pd

In [2]:
dbDetails = {
    "driver": "postgresql",
    "username": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "TML"
}
dbDetails["uri"] = '%(driver)s://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s' % dbDetails


In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [4]:
postgres_engine = create_engine(dbDetails["uri"], echo=False)
Sessions= sessionmaker(bind=postgres_engine)
session = Sessions()

In [5]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Numeric,Boolean
Base = declarative_base()

In [6]:
class PrimaryStockData(Base):
    # table properties
    """NA"""
    __tablename__ = 'primary_stock_data_v2'
    # table columns
    source_tag = Column(Integer, primary_key=True, autoincrement=False)
    status_date = Column(Date, primary_key=True, autoincrement=False)
    sku_code = Column(String(length=256), primary_key=True)
    sc_node_code = Column(String(length=256), primary_key=True)
    sales_channel_code = Column(String(length=256), primary_key=True)
    field_of_operation = Column(String(length=256), primary_key=True)
    stock_type = Column(String, primary_key=True)
    storage_loc = Column(String(length=256))
    batch_code = Column(String(length=256), primary_key=True)
    date_of_mfg = Column(Date)
    date_of_expiry = Column(Date)
    stock_qty = Column(Numeric(precision=38, scale=4))
    stock_value = Column(Numeric(precision=38, scale=4))
    uom = Column(String(length=256), default="NA")

class NodeMaster(Base):
    """Contains information about all the nodes for a given client in the system.
    User can put node code, description and other relevant information about a node in this table."""
    # table properties
    __tablename__ = 'node_master'

    # table columns
    source_tag = Column(Integer, primary_key=True, autoincrement=False)
    node_code  = Column(String(length=256), primary_key=True, autoincrement=False)
    node_description = Column(String(length=256))
    node_tier = Column(Integer)
    node_geo_code  = Column(String(length=256))  # Pin code or Plus Code
    status_code = Column(String(length=256), nullable=True)
    sales_channel = Column(String(length=256))
    region_code = Column(String(length=256))
    region_description = Column(String(length=256))
    state_code = Column(String(length=256))
    state_desc = Column(String(length=256))
    city_code = Column(String(length=256))
    city_desc = Column(String(length=256))
    pincode = Column(Integer)

In [7]:
#this is for the first time to setup table in DB
Base.metadata.create_all(postgres_engine)
#once created restore data in DB

In [8]:
date_start = '2019-07-01'
date_end ='2019-07-02'

In [9]:
sq = session.query(PrimaryStockData) \
    .filter(PrimaryStockData.status_date.between(cleft=date_start, cright=date_end)
            ,(PrimaryStockData.stock_type == "SFLT")
            ,(PrimaryStockData.sales_channel_code.in_(["CV","ALL"]))
            , (PrimaryStockData.field_of_operation.in_(["D"]))
            )

In [10]:
sq = sq.join(NodeMaster, NodeMaster.node_code == PrimaryStockData.sc_node_code)

In [11]:
print (sq.statement)

SELECT primary_stock_data_v2.source_tag, primary_stock_data_v2.status_date, primary_stock_data_v2.sku_code, primary_stock_data_v2.sc_node_code, primary_stock_data_v2.sales_channel_code, primary_stock_data_v2.field_of_operation, primary_stock_data_v2.stock_type, primary_stock_data_v2.storage_loc, primary_stock_data_v2.batch_code, primary_stock_data_v2.date_of_mfg, primary_stock_data_v2.date_of_expiry, primary_stock_data_v2.stock_qty, primary_stock_data_v2.stock_value, primary_stock_data_v2.uom 
FROM primary_stock_data_v2 JOIN node_master ON node_master.node_code = primary_stock_data_v2.sc_node_code 
WHERE primary_stock_data_v2.status_date BETWEEN :status_date_1 AND :status_date_2 AND primary_stock_data_v2.stock_type = :stock_type_1 AND primary_stock_data_v2.sales_channel_code IN (:sales_channel_code_1, :sales_channel_code_2) AND primary_stock_data_v2.field_of_operation IN (:field_of_operation_1)


In [12]:
sq = sq.filter(NodeMaster.node_tier != 2)

In [13]:
groubby_cols = list()
agg_cols = list()

In [14]:
### for StockPrimary
g1 = ['status_date','sku_code','sc_node_code']
## for node_tier
g2 = ['node_tier']

In [15]:
groubby_cols =  groubby_cols + [getattr(PrimaryStockData,col) for col in g1]

In [16]:
groubby_cols =  groubby_cols + [getattr(NodeMaster,col) for col in g2]

In [17]:
groubby_cols

In [18]:
sq = sq.group_by(*groubby_cols)

In [26]:
from sqlalchemy import func,and_,distinct

In [27]:
?distinct

In [20]:
# StockPrimary
ag1 = {"stock_qty" : {'sum' : "stock_qty"},
       "date_of_mfg" : {'min' : "min_date_of_mfg"}}

In [21]:
for col, agg_fn_info in ag1.items():
    print (col,agg_fn_info)
    column_id = getattr(PrimaryStockData, col)
    for function_name, label in agg_fn_info.items():
        function = getattr(func, function_name)
        agg_cols.append(function(column_id).label(label))

stock_qty {'sum': 'stock_qty'}
date_of_mfg {'min': 'min_date_of_mfg'}


In [22]:
agg_cols

In [23]:
sq = sq.with_entities(*groubby_cols, *agg_cols)
print (sq)

SELECT primary_stock_data_v2.status_date AS primary_stock_data_v2_status_date, primary_stock_data_v2.sku_code AS primary_stock_data_v2_sku_code, primary_stock_data_v2.sc_node_code AS primary_stock_data_v2_sc_node_code, node_master.node_tier AS node_master_node_tier, sum(primary_stock_data_v2.stock_qty) AS stock_qty, min(primary_stock_data_v2.date_of_mfg) AS min_date_of_mfg 
FROM primary_stock_data_v2 JOIN node_master ON node_master.node_code = primary_stock_data_v2.sc_node_code 
WHERE primary_stock_data_v2.status_date BETWEEN %(status_date_1)s AND %(status_date_2)s AND primary_stock_data_v2.stock_type = %(stock_type_1)s AND primary_stock_data_v2.sales_channel_code IN (%(sales_channel_code_1)s, %(sales_channel_code_2)s) AND primary_stock_data_v2.field_of_operation IN (%(field_of_operation_1)s) AND node_master.node_tier != %(node_tier_1)s GROUP BY primary_stock_data_v2.status_date, primary_stock_data_v2.sku_code, primary_stock_data_v2.sc_node_code, node_master.node_tier


In [24]:
df = pd.read_sql(sq.statement, session.bind)


In [25]:
df.head()

,status_date,sku_code,sc_node_code,node_tier,stock_qty,min_date_of_mfg
0,2019-07-01,20671936000R,2001,1,2.0,2017-03-27
1,2019-07-01,20731158000R,3001,1,1.0,2017-04-29
2,2019-07-01,20732258000R,3001,1,4.0,2016-06-22
3,2019-07-01,20732755100R,3001,1,2.0,2016-08-29
4,2019-07-01,20804332000R,3001,1,11.0,2019-06-29
